In [16]:
import pint.models as model
import pint.toa as toa
import pint.logging
import pint.fitter
from pint.modelutils import model_ecliptic_to_equatorial
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, ICRS, Galactic
import uncertainties as unc
import table_utils as tu
import glob

import pint.derived_quantities as dq
#See num2tex (https://github.com/AndrewChap/num2tex)
# > pip install num2tex
from num2tex import num2tex

pint.logging.setup(level="ERROR")

2

In [2]:
psrs = [
    "J0032+6946",
    "J0141+6303",
    "J0214+5222",
    "J0415+6111",
    "J0636+5128",
    "J0957-0619",
    "J1239+3239",
    "J1327+3423",
    "J1434+7257",
    "J1505-2524",
    "J1530-2114",
    "J1816+4510",
    "J1913+3732",
    "J1929+6630",
    "J1930+6205",
    "J2104+2830",
    "J2115+6702",
    "J2145+2158",
    "J2210+5712",
    "J2326+6243",
    "J2354-2250"
]

In [12]:
# Auto-populate rotational/timing params table
pars = sorted(glob.glob(f"data/*_fiore+23.par"))

f0ve = {}
f1ve = {}

for par in pars:
    with open(par, 'r') as infile:
        f2 = "--"
        for l in infile.readlines():
            if l.startswith("PSRJ"):
                psr     = l.split()[1]
                psr_tex = psr.replace('-','$-$')
            elif l.startswith("F0"):
                F0      = float(l.split()[1])
                F0_err  = float(l.split()[3])
                f0      = tu.ufve(F0, F0_err)
                if len(f0.split(".")[0]) == 1:
                    f0 = f"\dphn{f0}"
                elif len(f0.split(".")[0]) == 2:
                    f0 = f"\phn{f0}"
                f0ve[psr] = [F0*u.Hz, F0_err*u.Hz]
            elif l.startswith("F1"):
                F1      = float(l.split()[1])
                F1_err  = float(l.split()[3])
                f1      = tu.ufve(F1, F1_err).replace("^","$^").replace("}","}$")
                f1ve[psr] = [F1*u.Hz*u.s**-1, F1_err*u.Hz*u.s**-1]
            elif l.startswith("F2"):
                F2      = float(l.split()[1])
                F2_err  = float(l.split()[3])
                f2      = tu.ufve(F2, F2_err).replace("^","$^").replace("}","}$")
            elif l.startswith("PEPOCH"):
                epoch   = l.split()[1]
            elif l.startswith("START"):
                start   = float(l.split()[1])
            elif l.startswith("FINISH"):
                finish  = float(l.split()[1])
            elif l.startswith("TRES"):
                rms_us  = round(float(l.split()[1]), 1)
                if len(str(rms_us).split(".")[0]) == 1:
                    rms_us = f"\\tphn{rms_us}"
                elif len(str(rms_us).split(".")[0]) == 2:
                    rms_us = f"\dphn{rms_us}"
                elif len(str(rms_us).split(".")[0]) == 3:
                    rms_us = f"\phn{rms_us}"
            elif l.startswith("NTOA"):
                ntoa    = l.split()[1]
                if len(ntoa) == 2:
                    ntoa = f"\dphn{ntoa}"
                elif len(ntoa) == 3:
                    ntoa = f"\phn{ntoa}"
            elif l.startswith("CHI2R"):
                chi2r   = float(l.split()[1])
                efac    = round(np.sqrt(chi2r), 4)
            else:
                pass
        span = f"{int(start)}--{int(finish)}"
    # '\' is a special python character, but prefixing with '\' turns it into a normal character print('\\') -> '\'
    #out_str = f"{psr_tex} & {f0} & {f1} & {f2} & {epoch} & {span} & {rms_us:.1f} & {ntoa} & {chi2r:.2f} \\\\"
    out_str = f"{psr_tex} & {f0} & {f1} & {epoch} & {span} & {rms_us} & {ntoa} & {chi2r:.2f} \\\\"
    print(out_str.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("$-$-","--"))

J0032+6946 & \phn27.171118951999(1) & $-$2.65006(3)$\times$10$^{-15}$ & 57606 & 55169--58303 & \dphn19.1 & 1430 & 1.17 \\
J0141+6303 & \phn21.42232413061(4) & $-$7.65(2)$\times$10$^{-16}$ & 57312 & 57072--57789 & \dphn75.1 & \phn122 & 1.05 \\
J0214+5222 & \phn40.691271130935(5) & $-$4.9002(7)$\times$10$^{-16}$ & 56974 & 55353--58594 & \dphn78.3 & \phn951 & 1.11 \\
J0415+6111 & \dphn2.27174929826(6) & $-$2.8(4)$\times$10$^{-16}$ & 57234 & 57071--57397 & \phn609.4 & \dphn38 & 1.08 \\
J0636+5128 & 348.55922629556(2) & $-$4.189(6)$\times$10$^{-16}$ & 57277 & 56027--57397 & \tphn1.5 & \phn803 & 1.54 \\
J0957$-$0619 & \dphn0.58014345894(2) & $-$5(1)$\times$10$^{-17}$ & 57220 & 57071--57369 & \phn767.0 & \dphn67 & 1.14 \\
J1239+3239 & 212.71644800103(2) & $-$1.752(5)$\times$10$^{-16}$ & 57733 & 56054--59412 & \dphn21.5 & \phn283 & 1.28 \\
J1327+3423 & \phn24.0890076960438(9) & $-$7.514(3)$\times$10$^{-17}$ & 58334 & 57079--59055 & \tphn2.7 & 1575 & 1.39 \\
J1434+7257 & \phn23.957175000920(1) 

In [14]:
# Auto-populate derived common properties table

for psr in psrs:
    par_path = f"data/{psr}_fiore+23.par"
    psr_tex = psr.replace('-','$-$')
    
    f0,f0err = f0ve[psr]
    f1,f1err = f1ve[psr]
            
    # Calculate derived quantities with PINT
    p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
    p_tex = tu.ufve(p.value,perr.value)
    pd_tex = tu.ufve(pd.value,pderr.value)
    age = f"{num2tex(dq.pulsar_age(f0,f1).value):.1e}" # yr
    bsurf = f"{num2tex(dq.pulsar_B(f0,f1).value):.1e}" # G
    edot = f"{num2tex(dq.pulsar_edot(f0,f1).value):.1e}" # erg/s
    
    out_str = f"{psr_tex} & {p_tex} & {pd_tex} & {age} & {bsurf} & {edot} \\\\"
    
    print(out_str.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("^","$^").replace("}","}$"))

J0032+6946 & 0.036803784259552(1) & 3.58955(4)$\times$10$^{-18}$ & 1.6$\times$10$^{8}$ & 1.2$\times$10$^{10}$ & 2.8$\times$10$^{33}$ \\
J0141+6303 & 0.04668027586097(8) & 1.666(4)$\times$10$^{-18}$ & 4.4$\times$10$^{8}$ & 8.9$\times$10$^{9}$ & 6.5$\times$10$^{32}$ \\
J0214+5222 & 0.024575295197396(3) & 2.9595(4)$\times$10$^{-19}$ & 1.3$\times$10$^{9}$ & 2.7$\times$10$^{9}$ & 7.9$\times$10$^{32}$ \\
J0415+6111 & 0.44018941736(1) & 5.5(8)$\times$10$^{-17}$ & 1.3$\times$10$^{8}$ & 1.6$\times$10$^{11}$ & 2.5$\times$10$^{31}$ \\
J0636+5128 & 0.0028689528910994(2) & 3.448(5)$\times$10$^{-21}$ & 1.3$\times$10$^{10}$ & 1.0$\times$10$^{8}$ & 5.8$\times$10$^{33}$ \\
J0957$-$0619 & 1.72371158304(5) & 1.5(3)$\times$10$^{-16}$ & 1.8$\times$10$^{8}$ & 5.1$\times$10$^{11}$ & 1.2$\times$10$^{30}$ \\
J1239+3239 & 0.0047010939182060(4) & 3.87(1)$\times$10$^{-21}$ & 1.9$\times$10$^{10}$ & 1.4$\times$10$^{8}$ & 1.5$\times$10$^{33}$ \\
J1327+3423 & 0.041512710387163(2) & 1.2948(5)$\times$10$^{-19}$ & 5.1$\

In [15]:
# Auto-populate coordinates/DM table
rave = {}
decve = {}
dmve = {}

for psr in psrs:    
    par_path = f"data/{psr}_fiore+23.par"
    psr_tex = psr.replace('-','$-$')
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("RAJ"):
                ra_str = l.split()[1]
                ra_err = float(l.split()[3])
                rave[psr] = [ra_str,ra_err]
            elif l.startswith("DECJ"):
                dec_str = l.split()[1].strip("+")
                dec_err = float(l.split()[3])
                decve[psr] = [dec_str,dec_err]
            elif l.startswith("DM "):
                dm_val = float(l.split()[1])
                try:
                    dm_err = float(l.split()[3])
                    dmve[psr] = [dm_val,dm_err]
                    dm_tex = tu.ufve(dm_val,dm_err)
                except IndexError:
                    dmve[psr] = [dm_val,None]
                    dm_tex = f"{dm_val:.4f}"
    
    eqcoord = SkyCoord(ra_str,dec_str,frame=ICRS,unit=(u.hourangle, u.deg))

    ra = tu.format_ra(eqcoord,ra_err)
    dec = tu.format_dec(eqcoord,dec_err)
    
    gcoord = eqcoord.transform_to(Galactic)
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,dm_val)
    if dmdist_ne < 10.:
        dmdist_ne = f"\phn{dmdist_ne:.1f}"
    else:
        dmdist_ne = f"{dmdist_ne:.1f}"
    if dmdist_ymw < 10.:
        dmdist_ymw = f"\phn{dmdist_ymw:.1f}"
    else:
        dmdist_ymw = f"{dmdist_ymw:.1f}"
    gl = f"{gcoord.l.deg:.2f}"
    gb = f"{gcoord.b.deg:.2f}".replace('-','$-$')
    print(f"{psr_tex} & {ra} & {dec} & {dm_tex} & {gl} & {gb} & {dmdist_ne} & {dmdist_ymw} \\\\")#.replace("^","$^").replace("}","}$"))

J0032+6946 & $00^{\rm h}\, 32^{\rm m}\, 41\, \fs2477(3)$ & $+69\arcdeg\, 46\arcmin\, 28\, \farcs047(2)$ & 79.9988(2) & 121.30 & 6.96 & 2.8 & 2.3 \\
J0141+6303 & $01^{\rm h}\, 41^{\rm m}\, 45\, \fs761(1)$ & $+63\arcdeg\, 03\arcmin\, 49\, \farcs445(9)$ & 272.762(2) & 128.60 & 0.75 & 44.3 & 8.8 \\
J0214+5222 & $02^{\rm h}\, 14^{\rm m}\, 55\, \fs2745(2)$ & $+52\arcdeg\, 22\arcmin\, 40\, \farcs907(3)$ & 22.0369(3) & 135.63 & $-$8.42 & 1.0 & 1.2 \\
J0415+6111 & $04^{\rm h}\, 15^{\rm m}\, 51\, \fs63(5)$ & $+61\arcdeg\, 11\arcmin\, 51\, \farcs8(3)$ & 70.8(1) & 145.15 & 7.49 & 2.3 & 1.8 \\
J0636+5128 & $06^{\rm h}\, 36^{\rm m}\, 04\, \fs84748(7)$ & $+51\arcdeg\, 28\arcmin\, 59\, \farcs961(1)$ & 11.1075 & 163.91 & 18.64 & 0.5 & 0.2 \\
J0957$-$0619 & $09^{\rm h}\, 57^{\rm m}\, 08\, \fs12(2)$ & $-06\arcdeg\, 19\arcmin\, 37\, \farcs5(9)$ & 27.3(1) & 244.83 & 36.20 & 1.2 & 2.5 \\
J1239+3239 & $12^{\rm h}\, 39^{\rm m}\, 27\, \fs3140(1)$ & $+32\arcdeg\, 39\arcmin\, 23\, \farcs379(2)$ & 16.8590(1) & 14

In [7]:
# Auto-populate proper motion, intrinsic P-dot table
psr_names = ['J0214+5222','J0636+5128','J1327+3423','J1434+7257','J1816+4510']
for psr in psr_names:
    par_path = f"data/{psr}_fiore+23.par" 
    
    psr_tex = psr.replace('-','$-$')
    ra_str, ra_err   = rave[psr]
    dec_str, dec_err = decve[psr]
    dm, dm_err = dmve[psr]
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("PMRA"):
                pmra_val = float(l.split()[1])
                pmra_err = float(l.split()[3])
                pmra = f"{tu.ufve(pmra_val,pmra_err)}".replace("-","$-$")
            elif l.startswith("PMDEC"):
                try:
                    pmdec_val = float(l.split()[1])
                    pmdec_err = float(l.split()[3])
                except:
                    pmdec_val = -0.2
                    pmdec_err = 1.9
                pmdec = f"{tu.ufve(pmdec_val,pmdec_err)}".replace("-","$-$")
            else:
                pass
            
    eqcoord = SkyCoord(ra_str,dec_str,frame=ICRS,unit=(u.hourangle, u.deg))
    gcoord = eqcoord.transform_to(Galactic)
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,dm)
    
    pmtot,pmtoterr = tu.PMtot_err(pmra_val*u.mas/u.yr,pmra_err*u.mas/u.yr,pmdec_val*u.mas/u.yr,pmdec_err*u.mas/u.yr)

    for ii,dmdist in enumerate([dmdist_ne,dmdist_ymw]):
        dd,de = (dmdist,dmdist*0.3)*u.kpc
        dist = tu.ufve(dd.value,de.value)
        vt,vterr = tu.Vtrans_err(pmtot,pmtoterr,dd,de)
        vt_str = tu.ufve(vt.value,vterr.value)

        f0,f0err = f0ve[psr]
        f1,f1err = f1ve[psr]
        p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
        pdshk = dq.shklovskii_factor(pmtot,dd)*p.decompose()
        pds = f"{pdshk*1e21:.2f}"
        pdgal = tu.pd_gal(p,gcoord,dd).decompose()
        pdg = f"{pdgal*1e21:.2f}".replace("-","$-$")

        # Am I doing the math here correctly?
        pdint = pd-(pdshk+pdgal)
        pdi = f"{pdint.value*1e19:.2f}"

        # Re-derive age, bsurf, edot with intrinsic P-dot
        f1 = -1.0*pdint/p**2
        age = f"{dq.pulsar_age(f0,f1).value*1e-9:.1f}" # Gyr
        bsurf = f"{dq.pulsar_B(f0,f1).value*1e-9:.1f}" # 10^9 G
        edot = f"{dq.pulsar_edot(f0,f1).value*1e-33:.1f}" # 10^33 erg/s

        if ii == 0:
            print(f"{psr_tex} & {pmra} & {pmdec} & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\".replace(" \\times ","$\\times$").replace("^","$^").replace("}","}$"))
        elif ii == 1:
            print(f" & & & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\".replace(" \\times ","$\\times$").replace("^","$^").replace("}","}$"))

J0214+5222 & 10(1) & $-$0(2) & 1.0(3) & 5(2)$\times$10$^{1}$ & $-$0.04 & 6.09 & 2.90 & 2.7 & 1.3 & 0.8 \\
 & & & 1.2(3) & 5(2)$\times$10$^{1}$ & $-$0.02 & 6.85 & 2.89 & 2.7 & 1.3 & 0.8 \\
J0636+5128 & 1.7(4) & $-$3.2(8) & 0.5(1) & 8(3) & 0.03 & 0.04 & 0.03 & 0.1 & 13.3 & 5.7 \\
 & & & 0.21(6) & 4(1) & 0.01 & 0.02 & 0.03 & 0.1 & 13.1 & 5.8 \\
J1327+3423 & $-$7.8(1) & 5.5(2) & 0.5(1) & 22(6) & $-$5.84 & 4.40 & 1.31 & 2.4 & 5.0 & 0.1 \\
 & & & 0.3(1) & 16(5) & $-$4.93 & 3.18 & 1.31 & 2.4 & 5.0 & 0.1 \\
J1434+7257 & $-$4.5(9) & $-$7.6(6) & 0.7(2) & 30(9) & $-$4.96 & 5.63 & 5.48 & 4.8 & 1.2 & 0.3 \\
 & & & 1.0(3) & 4(1)$\times$10$^{1}$ & $-$5.87 & 7.69 & 5.47 & 4.8 & 1.2 & 0.3 \\
J1816+4510 & 4.0(4) & $-$4.4(3) & 2.4(7) & 7(2)$\times$10$^{1}$ & $-$0.84 & 0.66 & 0.43 & 0.4 & 1.2 & 52.5 \\
 & & & 4(1) & 1.2(4)$\times$10$^{2}$ & $-$1.42 & 1.19 & 0.43 & 0.4 & 1.2 & 52.5 \\


In [8]:
# Auto-populate ELL1 binary table
binary_psrs_ell1 = ['J0636+5128','J1239+3239','J1816+4510']
for bp in binary_psrs_ell1:
    par_path = f"data/{bp}_fiore+23.par"
    
    psr_tex = bp.replace('-','$-$')
    with open(par_path, 'r') as infile:
        #pbdot_tex = "\ldots"
        for l in infile.readlines():
            if l.startswith("A1"):
                a1_val = float(l.split()[1])
                a1_err = float(l.split()[3])
                a1_tex = tu.ufve(a1_val,a1_err)
            elif l.startswith("PB "):
                pb_val = float(l.split()[1])
                pb_err = float(l.split()[3])
                pb_tex = tu.ufve(pb_val,pb_err)
            elif l.startswith("FB0"):
                fb_val = float(l.split()[1])
                fb_err = float(l.split()[3])
                pb_val = (1./(fb_val*u.Hz)).to(u.day).value
                pb_err = ((fb_err*u.Hz)*(pb_val*u.day)**2).value
                pb_tex = tu.ufve(pb_val,pb_err)
#             elif l.startswith("PBDOT"):
#                 pbdot_val = float(l.split()[1])
#                 pbdot_err = float(l.split()[3])
#                 pbdot_tex = tu.ufve(pbdot_val,pbdot_err)
            elif l.startswith("TASC"):
                tasc_val = float(l.split()[1])
                tasc_err = float(l.split()[3])
                tasc_tex = tu.ufve(tasc_val,tasc_err)
            elif l.startswith("EPS1"):
                eps1_val = float(l.split()[1])
                eps1_err = float(l.split()[3])
                eps1_tex = tu.ufve(eps1_val,eps1_err)
            elif l.startswith("EPS2"):
                eps2_val = float(l.split()[1])
                eps2_err = float(l.split()[3])
                eps2_tex = tu.ufve(eps2_val,eps2_err)
            else:
                pass
    
    # Derive mass function and Mc,min
    fm = dq.mass_funct(pb_val*u.day,a1_val*u.lightsecond)
    fm_tex = f"{num2tex(fm.value):.2e}" # Msun
    mc = dq.companion_mass(pb_val*u.day,a1_val*u.lightsecond, i=90.0*u.deg, mp=1.4 * u.solMass)
    mc_tex = f"{mc.value:.3f}" # Msun

    #outstr = f"{psr_tex} & {pb_tex} & {pbdot_tex} & {a1_tex} & {tasc_tex} & {eps1_tex} & {eps2_tex} & {fm_tex} & {mc_tex} \\\\"
    outstr = f"{psr_tex} & {pb_tex} & {a1_tex} & {tasc_tex} & {eps1_tex} & {eps2_tex} & {fm_tex} & {mc_tex} \\\\"
    print(outstr.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("^","$^").replace("}","}$"))

J0636+5128 & 0.066551342603405(4) & 0.0089858(1) & 57277.0161641(6) & 1(2)$\times$10$^{-5}$ & $-$1(2)$\times$10$^{-5}$ & 1.76$\times$10$^{-7}$ & 0.007 \\
J1239+3239 & 4.085401710(6) & 2.371127(2) & 57730.0802277(9) & 5(2)$\times$10$^{-6}$ & 0(2)$\times$10$^{-6}$ & 8.58$\times$10$^{-4}$ & 0.126 \\
J1816+4510 & 0.360893486752638(1) & 0.595390(2) & 56945.0913684(2) & $-$9(3)$\times$10$^{-6}$ & $-$1(2)$\times$10$^{-6}$ & 1.74$\times$10$^{-3}$ & 0.162 \\


In [9]:
# Auto-populate DD binary table
binary_psrs_dd = ['J0032+6946','J0214+5222']

for bp in binary_psrs_dd:
    par_path = f"data/{bp}_fiore+23.par"
    
    psr_tex = bp.replace('-','$-$')
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("A1"):
                a1_val = float(l.split()[1])
                a1_err = float(l.split()[3])
                a1_tex = tu.ufve(a1_val,a1_err)
            elif l.startswith("PB "):
                pb_val = float(l.split()[1])
                pb_err = float(l.split()[3])
                pb_tex = tu.ufve(pb_val,pb_err)
            elif l.startswith("T0"):
                t0_val = float(l.split()[1])
                t0_err = float(l.split()[3])
                t0_tex = tu.ufve(t0_val,t0_err)
            elif l.startswith("ECC"):
                ecc_val = float(l.split()[1])
                ecc_err = float(l.split()[3])
                ecc_tex = tu.ufve(ecc_val,ecc_err)
            elif l.startswith("OM"):
                om_val = float(l.split()[1])
                om_err = float(l.split()[3])
                om_tex = tu.ufve(om_val,om_err)
            else:
                pass

    # Derive mass function and Mc,min
    fm = dq.mass_funct(pb_val*u.day,a1_val*u.lightsecond)
    fm_tex = f"{num2tex(fm.value):.4e}" # Msun
    mc = dq.companion_mass(pb_val*u.day,a1_val*u.lightsecond, i=90.0*u.deg, mp=1.4 * u.solMass)
    mc_tex = f"{mc.value:.2f}" # Msun
    
    outstr = f"{psr_tex} & {pb_tex} & {a1_tex} & {t0_tex} & {ecc_tex} & {om_tex} & {fm_tex} & {mc_tex} \\\\"
    print(outstr.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("^","$^").replace("}","}$"))

J0032+6946 & 527.621324(2) & 178.674771(3) & 57142.972(2) & 0.00053181(3) & 147.525(2) & 2.2000$\times$10$^{-2}$ & 0.42 \\
J0214+5222 & 512.039768(2) & 174.565749(4) & 56638.6454(8) & 0.00532802(6) & 210.5907(5) & 2.1785$\times$10$^{-2}$ & 0.42 \\


In [10]:
# Auto-populate miscellaneous parameters table
for psr in psrs:    
    include=False
    par_path = f"data/{psr}_fiore+23.par"
    
    f2_tex = '\ldots'
    px_tex = '\ldots'
    fb1_tex = '\ldots'
    fb2_tex = '\ldots'
    xdot_tex = '\ldots'
    fd1_tex = '\ldots'
    
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("F2"):
                f2_val = float(l.split()[1])
                f2_err = float(l.split()[3])
                f2_tex = tu.ufve(f2_val, f2_err)
                include = True
            elif l.startswith("PX"):
                px_val = float(l.split()[1])
                try:
                    px_err = float(l.split()[3])
                except:
                    px_val = 1.74816089886028 # J0636 has PX set to zero
                    px_err = 1.6203
                px_tex = tu.ufve(px_val, px_err)
                include = True
            elif l.startswith("FB1"):
                fb1_val = float(l.split()[1])
                fb1_err = float(l.split()[3])
                fb1_tex = tu.ufve(fb1_val, fb1_err)
                include = True
            elif l.startswith("FB2"):
                fb2_val = float(l.split()[1])
                fb2_err = float(l.split()[3])
                fb2_tex = tu.ufve(fb2_val, fb2_err)
                include = True
            elif l.startswith("XDOT"):
                xdot_val = float(l.split()[1])
                xdot_err = float(l.split()[3])
                xdot_tex = tu.ufve(xdot_val, xdot_err)
                include = True
            elif l.startswith("FD1"):
                fd1_val = float(l.split()[1])
                fd1_err = float(l.split()[3])
                fd1_tex = tu.ufve(fd1_val, fd1_err)
                include = True
                
    if include==True:
        psr_tex = psr.replace('-','$-$')

        outstr = f"{psr_tex} & {f2_tex} & {px_tex} & {fb1_tex} & {fb2_tex} & {xdot_tex} & {fd1_tex} \\\\"
        print(outstr.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("^","$^").replace("}","}$"))

J0636+5128 & \ldots & 2(2) & $-$1.9(5)$\times$10$^{-19}$ & $-$2.5(8)$\times$10$^{-27}$ & \ldots & 2.9(2)$\times$10$^{-5}$ \\
J1327+3423 & \ldots & 4(1) & \ldots & \ldots & \ldots & 0.000162(2) \\
J1816+4510 & $-$7.1(7)$\times$10$^{-26}$ & \ldots & 3.3(8)$\times$10$^{-21}$ & 5(2)$\times$10$^{-29}$ & $-$1.0(2)$\times$10$^{-13}$ & \ldots \\
